In [ ]:
pip install mysql-connector-python

In [4]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector
import streamlit as st

from datetime import datetime

In [63]:
# Cargar csv
df_combustibles = pd.read_csv("../bin/listado_combustibles.csv")
df_concesionarios = pd.read_csv("../bin/listado_concesionarios.csv")
df_distintivo_ambiental = pd.read_csv("../bin/listado_distintivo_ambiental.csv")
df_marcas = pd.read_csv("../bin/listado_marcas.csv")
df_modelos = pd.read_csv("../bin/listado_modelos.csv")
df_provincias = pd.read_csv("../bin/listado_provincias.csv")
df_sobrealimentacion = pd.read_csv("../bin/listado_sobrealimentacion.csv")
df_traccion = pd.read_csv("../bin/listado_traccion.csv")
df_prestaciones = pd.read_csv("../bin/prestaciones.csv")
df_prestaciones

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,...,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo,id_combustible,id_tipo_cambio
0,1,58000,1,1,12.0,19990.0,37250.0,4633.0,1811.0,1429.0,...,8.7,11,2018,118.0,4.0,1,8.0,1,1,1
1,1,34680,1,2,12.0,12990.0,19300.0,3571.0,1627.0,1488.0,...,13.8,12,2021,NaN,3.0,2,19.0,2,2,1
2,1,66933,2,1,12.0,17390.0,26650.0,4377.0,1806.0,1590.0,...,11.9,4,2017,103.0,4.0,2,50.0,3,1,1
3,2,159000,3,1,12.0,8000.0,22425.0,4702.0,1809.0,1499.0,...,11.4,1,2017,99.0,4.0,2,52.0,4,1,1
4,1,62695,4,3,12.0,22990.0,44700.0,4545.0,1805.0,1685.0,...,10.9,7,2022,NaN,4.0,2,48.0,5,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33043,4,28000,613,2,24.0,79500.0,82000.0,4713.0,1897.0,1676.0,...,4.9,12,2023,NaN,6.0,1,8.0,99,1,2
33044,21,12280,184,1,12.0,24000.0,NaN,NaN,NaN,NaN,...,NaN,8,2023,NaN,NaN,3,6.0,127,2,1
33045,1,18744,5,3,36.0,38990.0,57005.0,4684.0,1834.0,1667.0,...,9.2,8,2023,NaN,NaN,4,44.0,29,4,1
33046,1,101353,4,1,12.0,13490.0,NaN,NaN,NaN,NaN,...,NaN,8,2018,NaN,NaN,3,56.0,80,2,1


In [ ]:
# Conectar a la base de datos usando los valores almacenados en `secrets.toml`
def conectar_base_datos():
    conn = mysql.connector.connect(
        host=st.secrets["database"]["host"],
        user=st.secrets["database"]["user"],
        password=st.secrets["database"]["password"],
        database=st.secrets["database"]["database"]
    )
    return conn

In [65]:
# Funcion para insertar coches en la base de datos
def insertar_datos(df, nombre_tabla):
    # Conexión a la base df_cochedatos de MySQL
    conn = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "Whereisbotus1996",
        database = "coches_segunda_mano"
    )
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()
    # Obtener las columnas del DataFrame
    columnas = df.columns.tolist()
    # Convertir cada fila del DataFrame a una tupla
    valores = [tuple(row) for row in df.to_numpy()]

    # Crear la consulta SQL para insertar datos
    query = f"""
        INSERT INTO {nombre_tabla} ({', '.join(columnas)})
        VALUES ({', '.join(['%s'] * len(columnas))})
    """

    # Ejecutar la consulta para todas las filas
    cursor.executemany(query, valores)

    # Confirmar los cambios en la base de datos
    conn.commit()

    # Cerrar el cursor y la conexion
    cursor.close()
    conn.close()

    # Mensaje de exito
    print("Datos insertados correctamente")

# Llamar a la funcion para insertar los datos en la tabla
insertar_datos(df_prestaciones, 'prestaciones')

ProgrammingError: 1054 (42S22): Unknown column 'id_traccion' in 'field list'

In [ ]:
# Funcion para extraer datos
def extraer_datos(nombre_tabla):
    # Conexión a la base de datos de MySQL
    conn = conectar_base_datos()
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()

    # Crear la consulta SQL para seleccionar todos los datos de la tabla
    query = f"SELECT * FROM {nombre_tabla}"

    # Ejecutar la consulta y recuperar los datos
    cursor.execute(query)
    resultado = cursor.fetchall()

    # Obtener los nombres de las columnas de la tabla
    columnas = [desc[0] for desc in cursor.description]

    # Convertir los resultados a un DataFrame
    df = pd.DataFrame(resultado, columns=columnas)

    # Cerrar el cursor y la conexión
    cursor.close()
    conn.close()

    # Devolver el DataFrame
    return df

# Llamar a la función para extraer datos de la tabla
df_extraido = extraer_datos('marcas')
print(df_extraido)